# Graphs - Playground

Simple graph utilities (creation, reporting), and some graph algorithms.

In [1]:
import numpy as np

**Topological order** or **topological sort**: a sequence of vertex ids, such that for no directed edge A→B, B is given in the list before A. Is only possible in a DAG (in a graph without directed cycles). https://en.wikipedia.org/wiki/Topological_sorting

In [33]:
class Graph:    
    def __init__(self,nv=1,directed=False):
        '''Create empty graph'''
        self.adj = {}
        self.nv = nv
        self.directed = directed # Plug for the future
        for i in range(self.nv):
            self.adj[i] = []
            
    def n(self): return len(self.adj)
            
    def __str__(self):
        return f'Graph of {len(self.adj)} edges:\n'+'\n'.join([str(key)+':'+str(self.adj[key]) for key in self.adj])
    
    def add_edge(self,i,j):
        if i not in self.adj: 
            self.adj[i] = [j]
        else:                 
            self.adj[i].append(j)
        if not self.directed:
            if j not in self.adj: 
                self.adj[j] = [i]
            else:                 
                self.adj[j].append(i)
        else:
            if j not in self.adj: 
                self.adj[j] = []
                
    def add_edges(self,ts):
        '''Accepts a list of tuples'''
        for (i,j) in ts:
            self.add_edge(i,j)
                
    def adj_matrix(self):
        out = np.zeros((len(self.adj),len(self.adj)),dtype=int)
        for i in self.adj:
            for j in self.adj[i]:
                out[i,j] = 1
        return out
    
    def dfs(self,v=None,visited=None,path=None,topord=None,verbose=False):
        """Depth-first search; returns topological sorting."""
        if v is None: v=next(iter(self.adj.keys()))     # If needed, pick some random vertex as a root
        if visited is None: visited = [0]*len(self.adj) # If needed, mark all v as unvisited
        if path is None:    path = []
        if topord is None:  topord = []                 # Topological ordering, will be returned
        visited[v] = 1                                  # Mark current v as visited
        path += [v]
        if verbose: print(path)
        for i in self.adj[v]:                           # For all connections from the current vertex
            if visited[i]==0:                           # If they weren't yet visited, visit them
                topord = self.dfs(i,visited,path,topord=topord,verbose=verbose)
        return [v]+topord                               # Update topo-order when LEAVING the node (reverse postorder)
                
    def bfs(self,v=None,target=None,verbose=False):
        """Breadth-first exploration, looking for a distance from one vertex to another."""
        if v is None: v = next(iter(self.adj.keys()))   # If needed, pick some random vertex as a root
        q = [(v,0)]                                     # Queue of vertices to be visited, with their distances from the root
        visited = [0]*len(self.adj)                     # Not recursive, so mark all as unvisited
        while len(q)>0:                                 # While queue isn't empty
            (v,d) = q.pop(-1)                           # Pop the last element of the queue
            if v==target:
                return d
            if verbose: print(v,':',q)
            visited[v] =  1
            for i in self.adj[v]:
                if visited[i]==0 and ((i,d+1) not in q):
                    q.append((i,d+1))                 

In [34]:
# First, let's try it all with an undirected graph

g = Graph(directed=False)
g.add_edges([(0,1),(0,2),(1,3),(2,3),(3,5),(2,4),(4,5),(6,7)])
print(g)
#g.adj_matrix()
print('\ndfs:')
print('dfs output:',g.dfs(0,verbose=True))
print('\nbfs:')
print('bfs output:',g.bfs(verbose=True))
print(g.bfs(0,3))
print(g.bfs(3,0))
print(g.bfs(2,5))

Graph of 8 edges:
0:[1, 2]
1:[0, 3]
2:[0, 3, 4]
3:[1, 2, 5]
5:[3, 4]
4:[2, 5]
6:[7]
7:[6]

dfs:
[0]
[0, 1]
[0, 1, 3]
[0, 1, 3, 2]
[0, 1, 3, 2, 4]
[0, 1, 3, 2, 4, 5]
dfs output: [0, 1, 3, 2, 4, 5]

bfs:
0 : []
2 : [(1, 1)]
4 : [(1, 1), (3, 2)]
5 : [(1, 1), (3, 2)]
3 : [(1, 1), (3, 2)]
1 : [(1, 1), (3, 2)]
3 : [(1, 1)]
1 : []
bfs output: None
4
4
2


In [35]:
# Now, exactly same commands, but with a directed graph

g = Graph(directed=True)
g.add_edges([(0,1),(0,2),(1,3),(2,3),(3,5),(2,4),(4,5),(6,7)])
print(g)
#g.adj_matrix()
print('\ndfs:')
print('dfs output:',g.dfs(0,verbose=True))
print('\nbfs:')
print('bfs output:',g.bfs(verbose=True))
print(g.bfs(0,3))
print(g.bfs(3,0))
print(g.bfs(2,5))

Graph of 8 edges:
0:[1, 2]
1:[3]
2:[3, 4]
3:[5]
5:[]
4:[5]
6:[7]
7:[]

dfs:
[0]
[0, 1]
[0, 1, 3]
[0, 1, 3, 5]
[0, 1, 3, 5, 2]
[0, 1, 3, 5, 2, 4]
dfs output: [0, 2, 4, 1, 3, 5]

bfs:
0 : []
2 : [(1, 1)]
4 : [(1, 1), (3, 2)]
5 : [(1, 1), (3, 2)]
3 : [(1, 1)]
1 : []
bfs output: None
2
None
2


In [29]:
# Random graph
g = Graph(directed=True)
for i in range(30):
    j = np.random.randint(g.n())
    g.add_edge(j,i)
# print(g)
print(g.dfs(verbose=False))

[0]
